In [ ]:
# Setup
import os
import importlib

import spectralmatch.match.global_regression as global_regression_mod
import spectralmatch.match.local_block_adjustment as local_block_adjustment_mod
import spectralmatch.handlers as handlers_mod

importlib.reload(global_regression_mod)
importlib.reload(local_block_adjustment_mod)
importlib.reload(handlers_mod)

from spectralmatch.match.global_regression import global_regression
from spectralmatch.match.local_block_adjustment import local_block_adjustment
from spectralmatch.handlers import merge_rasters

# -------------------- Parameters
working_directory = os.path.join(os.getcwd(), "data_worldview3")
# This script is setup to perform matching on all tif files from a folder within the working directory called "input" e.g. working_directory/input/*.tif.

vector_mask_path = working_directory + "/Input/Masks.gpkg"

input_folder = os.path.join(working_directory, "Input")
global_folder = os.path.join(working_directory, "GlobalMatch")
local_folder = os.path.join(working_directory, "LocalMatch")

In [ ]:
# Global histogram matching
input_image_paths_array = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(".tif")]

matched_global_images_paths = global_regression(
    input_image_paths_array,
    global_folder,
    custom_mean_factor = 3, # Defualt 1; 3 often works better to 'move' the spectral mean of images closer together
    custom_std_factor = 1,
    # vector_mask_path=vector_mask_path,
    debug_logs=True,
    window_size=(512, 512),
    parallel=True,
    )

merge_rasters(
    matched_global_images_paths, # Rasters are layered with the last ones on top
    os.path.join(working_directory, "MatchedGlobalImages.tif"),
    window_size=(512, 512),
    )

In [ ]:
# Local histogram matching
global_image_paths_array = [os.path.join(global_folder, f) for f in os.listdir(global_folder) if f.lower().endswith(".tif")]

matched_local_images_paths = local_block_adjustment(
    global_image_paths_array,
    local_folder,
    number_of_blocks=100,
    debug_logs=True,
    window_size=(300,300),
    parallel=True,
    )

merge_rasters(
    matched_local_images_paths, # Rasters are layered with the last ones on top
    os.path.join(working_directory, "MatchedLocalImages.tif"),
    window_size=(300, 300),
    )

print("Done with global and local histogram matching")

In [ ]:
# Start from saved block maps
saved_reference_path = os.path.join(local_folder, "BlockReferenceMean", "BlockReferenceMean.tif")
saved_local_folder_path = os.path.join(local_folder, "BlockLocalMean")
saved_local_paths = [os.path.join(saved_local_folder_path, f) for f in os.listdir(saved_local_folder_path) if f.lower().endswith(".tif")]

new_local_folder = os.path.join(working_directory, "New_LocalMatch")

matched_local_images_paths = local_block_adjustment(
    global_image_paths_array,
    new_local_folder,
    number_of_blocks=100,
    debug_logs=True,
    window_size=(512,512),
    parallel=True,
    pre_computed_block_map_paths=(saved_reference_path, saved_local_paths)
    )

matched_local_images_paths = [os.path.join(new_local_folder, f) for f in os.listdir(new_local_folder) if f.lower().endswith(".tif")]

merge_rasters(
    matched_local_images_paths, # Rasters are layered with the last ones on top
    os.path.join(new_local_folder, "MatchedLocalImages.tif"),
    window_size=(512, 512),
    )


In [ ]:
import os

from spectralmatch.voronoi_centerline_seamline import voronoi_centerline_seamline

working_directory = os.path.join(os.path.dirname(os.path.abspath(__file__)), "example_data")
input_folder = os.path.join(working_directory, "Output", "LocalMatch", "Images")
input_image_paths_array = [os.path.join(input_folder, f) for f in os.listdir(input_folder) if f.lower().endswith(".tif")]

voronoi_centerline_seamline(
    input_image_paths_array,
    './example_data/Output/seamline_mask.gpkg')

# Statistics
# To visually see the difference make sure to merge input images so that their histograms match